## Kinetic model with finite ion concentration

In [449]:
import numpy as np
import pandas as pd

import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

## Encounter formation
### 1st-order rate for encounter complex formation

The (outer) encounter complex is represented by the reference ion into a cylinder or radius $r\simeq 1.5~{\rm nm}$ and with $z>z_{\rm Out}$. The first-order  (Smouchowski 3d) on-rate this encounter complex is $k_{+1}^{\rm Out} = 4 \pi D r$. At inifinite dilution and at $25^\circ$ the self-diffusion coefficient of ${\rm Na}^+$ is 
$$D = 13\times10^{-6} ~{\rm cm}^2/s = 13\times10^{2}~{\rm nm}^2/\mu s$$
from Li and Gregory (1974) Geochimica and Cosmochimica Acta 38:703

We should calculate the diffusion from the simulations as well, and check for modern values of $D$.


In [54]:
r = 1.5 # in nm
D = 13e+2 # in nm^2/us
kp1_firstorder = 6.28*D*r # in nm^3/us

In [55]:
# calculate the number concentration of ions for physio extracellular concentration
mol2con= lambda mol: mol*0.62 # in nm^{-3}
nc = mol2con(150/1000.) 

In [20]:
# we should have approx this many ions
vol = 7.9*7.9*(10.6-4) # in nm^3
vol*nc

38.307258

### 0th-order rate for complex formation

The zeroth-order rate for the formation of the encounter complex is then
$c_{\rm EC} k_{+1}^{\rm Out} $ and time scale of encounter complex formation is $$\tau_{\rm EC} \sim \frac{1}{c k_{+1}^{\rm Out}} $$

In [18]:
kp1_firstorder*nc # in 1/us

1138.878

In [454]:
#def fun_k1p(mmol):
#    return kp1_firstorder*mol2con(mmol/1000.)

p = figure(plot_width=300, plot_height=300, tools="pan,wheel_zoom,box_zoom,reset,crosshair")
p.line(mmol,1./(kp1_firstorder*mol2con(mmol/1000.)), line_width=2)
p.xaxis.axis_label = 'ion concentration (mM)'
p.yaxis.axis_label = 'time (us)'
p.title.text='encounter cpx formation'
show(p)


### encounter complex dissociation

We model the unbinding from the encounter complex with a capture probability 
$$\gamma = \frac{k_{01}}{k_{01} + k_{-1}}$$
where $k_{01}$ is the rate to go from the encounter complex to the entry state in the bimolecular model 
($\gamma$ should be calculated from the simulations) and express the unbinding from the encounter complex as
$$ 
k_{-1} = k_{01} \frac{1-\gamma}{\gamma}
$$


## Synthetic bimolecular model mimicking the real matrix

In [255]:
zs = np.linspace(-40,40,50)
fess = -40*(zs-1)/(zs**2+5)
p = figure(plot_width=400, plot_height=200, tools="pan,wheel_zoom,box_zoom,reset,crosshair")
p.line(zs,fess, line_width=2)
p.line(zs,-40*(zs-1.5)/(zs**2+5.5), line_width=1)
p.line(zs,-40*(zs-0.9)/(zs**2+4.5), line_width=1)

p.xaxis.axis_label = 'ion concentration (mM)'
p.yaxis.axis_label = 'EC formation time (us)'
show(p)

In [240]:
from scipy import linalg
import pyemma 
from pyemma import msm
import msmtools

In [241]:
#ratematrix in 1/us
# we use D in nm^2/us
dz = (zs[1]-zs[0])*0.1 # in nm

fwd = np.diagflat(np.exp(0.5*fess[:-1])*np.exp(-0.5*fess[1:]),k=1)
bkw = np.diagflat(np.exp(0.5*fess[1:])*np.exp(-0.5*fess[:-1]),k=-1)
K0 = D/dz**2*(fwd + bkw) 
K = K0 - np.diagflat(K0.sum(axis=1),k=0)

In [242]:
## transition matrix at tau = 1 ns
tau = 1./1000 # in us
T_micro = scipy.linalg.expm(K*tau)
msm_micro = pyemma.msm.markov_model(T_micro)

In [348]:
## define macrostates
intracell = np.where(zs<-30)[0]
extracell = np.where(zs>30)[0]
bound = np.where((fess-fess.min())<1)[0]

In [349]:
tau*msm_micro.mfpt(extracell,bound)

0.0028938505434246485

In [350]:
tau*msm_micro.mfpt(bound,extracell)

0.069839609495908853

In [351]:
tau*msm_micro.mfpt(bound, intracell)

7058.0860861816

### Coupled model

In [352]:
def coupled_ratematrix(K0, k1p_out, k1p_in, gamma):
    """
    couples rate matrix with reservoirs inside and outside
    """
    E0 = scipy.linalg.block_diag([0.], K0, [0.])
    E0[0,1] = k1p_out
    E0[1,0] = (1-gamma)/gamma* E0[1,2]
    E0[-1,-2] = k1p_in
    E0[-2,-1] = (1-gamma)/gamma* E0[-2,-3]
    E = E0 - np.diagflat(E0.sum(axis=1),k=0)
    return E

In [388]:
E = coupled_ratematrix(
    K0, 
    kp1_firstorder*mol2con(10/1000.), 
    kp1_firstorder*mol2con(150/1000.), 
    gamma = 0.7)

T_coupled = scipy.linalg.expm(E*tau)
msm_coupled = pyemma.msm.markov_model(T_coupled)

In [389]:
## define macrostates
intracell_coupled = 0
extracell_coupled = -1
bound_coupled = bound+1

In [399]:
## times for this concentrations
[tau*msm_coupled.mfpt(extracell_coupled,bound_coupled),
 tau*msm_coupled.mfpt(bound_coupled,extracell_coupled),
 tau*msm_coupled.mfpt(bound_coupled,intracell_coupled),
 tau*msm_coupled.mfpt(intracell_coupled,bound_coupled)]

[0.0080775437409698891,
 0.10711464761982506,
 7449.9923419093066,
 796.25916903247207]

In [444]:
def rates_coupled(conc_out, conc_in):
    """
    returns corrected rates and thermo for given inner and extra concentrations
    """
    E = coupled_ratematrix(
    K0, 
    kp1_firstorder*mol2con(conc_in/1000.), 
    kp1_firstorder*mol2con(conc_out/1000.), 
    gamma = 0.7)
    T_coupled = scipy.linalg.expm(E*tau)
    msm_coupled = pyemma.msm.markov_model(T_coupled)
    bound_fraction = msm_coupled.pi[bound_coupled].sum()
    rates = np.array([
        conc_out, conc_in,
        tau*msm_coupled.mfpt(extracell_coupled,bound_coupled),
        tau*msm_coupled.mfpt(bound_coupled,extracell_coupled),
        tau*msm_coupled.mfpt(bound_coupled,intracell_coupled),
        tau*msm_coupled.mfpt(intracell_coupled,bound_coupled),
        bound_fraction
    ])
    return rates

In [445]:
concs_out = 10**np.linspace(-1,3,20)
data = pd.DataFrame([rates_coupled(conc_out,10) for conc_out in concs_out],
            columns=['conc_out', 'conc_in','bind', 'unbind', 'egress', 'reflux', 'bound'])
source = ColumnDataSource(data)

data

,conc_out,conc_in,bind,unbind,egress,reflux,bound
0,0.100000,10.0,7.384892,0.096007,594736.280803,796.325406,0.008511
1,0.162378,10.0,4.549185,0.096011,368896.887036,796.299944,0.013708
2,0.263665,10.0,2.802820,0.096018,229956.740079,796.284263,0.021968
3,0.428133,10.0,1.727324,0.096029,144326.257378,796.274606,0.034929
4,0.695193,10.0,1.064982,0.096047,91592.395525,796.268659,0.054864
5,1.128838,10.0,0.657080,0.096076,59120.636948,796.264996,0.084601
6,1.832981,10.0,0.405874,0.096123,39120.768819,796.262741,0.126989
7,2.976351,10.0,0.251170,0.096201,26803.480018,796.261352,0.183660
8,4.832930,10.0,0.155895,0.096326,19218.347656,796.260496,0.253264
9,7.847600,10.0,0.097220,0.096530,14546.987715,796.259969,0.330371


In [455]:
p = figure(plot_width=400, 
           plot_height=300, 
           tools="pan,wheel_zoom,box_zoom,reset,crosshair",
           x_axis_type="log"
          )
p.line('conc_out','bound', line_width=2, source=source)
p.xaxis.axis_label = 'ion concentration (mM)'
p.yaxis.axis_label = 'occupation of bound state'
p.title.text='ion occupation'
show(p)

In [458]:
p = figure(plot_width=400, 
           plot_height=300, 
           tools="pan,wheel_zoom,box_zoom,reset,crosshair",
           x_axis_type="log",
           y_axis_type="log"
           
          )
p.line(data['conc_out'],data['bind'], line_width=2,  line_color='red',legend='bind')
p.line(data['conc_out'],data['unbind'], line_width=2, legend='unbind')

p.xaxis.axis_label = 'ion concentration (mM)'
p.yaxis.axis_label = 'time (us)'
p.title.text='times bound/extracell (bind/unbind)'

show(p)

In [459]:
p = figure(plot_width=400, 
           plot_height=300, 
           tools="pan,wheel_zoom,box_zoom,reset,crosshair",
           x_axis_type="log",
           y_axis_type="log"
           
          )
p.line(data['conc_out'],data['egress'], line_width=2, line_color='red',legend='egress')
#p.line('conc_out','reflux', line_width=2, source=source)

p.xaxis.axis_label = 'ion concentration (mM)'
p.yaxis.axis_label = 'time (us)'
show(p)